In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Lecture .csv

df = pd.read_csv("Camp_Market.csv", sep=';')

In [4]:
# Encodage One-Hot

data_encoded = pd.get_dummies(df, columns=['Education'])

#var Living_with
def love_status(status):
    if status == "Single" or status == "Widow" or status == "Divorced":
        return 'Single'
    elif status == "Together" or status == "Married":
        return 'In a relationship'
data_encoded["Living_with"] = df["Marital_Status"].apply(love_status)

#var Family_size
data_encoded["Family_size"] = data_encoded["Kidhome"] + np.where(
    (data_encoded["Marital_Status"] == "Together") | (data_encoded["Marital_Status"] == "Married"), 
    2, 
    1
)

# var Customer_for nombre de jours depuis que le client a commencé ses achats au magasin depuis la dernière date enregistrée
today = pd.Timestamp(datetime.date.today())
data_encoded["Customer_for"] = (today - pd.to_datetime(df["Dt_Customer"]))

# var Age
data_encoded["Age"] = date.today().year - data_encoded["Year_Birth"]

#var Spend
data_encoded["Spent"] = data_encoded[[col for col in df.columns if col.startswith('Mnt')]].sum(axis=1)

#var Is_parent
data_encoded["Is_parent"] = np.where((data_encoded["Kidhome"] > 0) | (data_encoded["Teenhome"] > 0), 1, 0)

#var Children
data_encoded["Children"] = data_encoded.Kidhome + data_encoded.Teenhome

# drop des colonnes non utilisées
data_encoded = data_encoded.drop(columns=["Dt_Customer", "Marital_Status", "Dt_Customer", "Year_Birth", "Kidhome", "Teenhome", "Living_with"])

In [5]:
# Calcul de la matrice de corrélation pour le dataframe encodé
corr_matrix_encoded = data_encoded.corr()

#Remplacement NaN par 0
data_encoded = data_encoded.fillna(0)